In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 9.0 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=0de51b92ad36219d260641c12f5dca24e707067c854fa48dc1bffd8f1b012ec4
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [2]:
! signate list
! signate files --competition-id=492
! signate download --competition-id=492

  competitionId  title                                                                  closing     prize            submitters
---------------  ---------------------------------------------------------------------  ----------  -------------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                   -                                  5174
             24  【練習問題】お弁当の需要予測                                           -                                  6344
             27  【練習問題】Jリーグの観客動員数予測                                    -                                  1433
            100  【練習問題】手書き文字認識                                             -           Knowledge               174
            102  【練習問題】タイタニックの生存予測                                     -           Knowledge              1064
            103  【練習問題】音楽ラベリング                                             -           Knowledge                61
            104  【練習問題】スパムメール分類                                     

In [3]:
# ディレクトリ内のデータの確認
%ls

adc.json                                       requirements.txt
category_names.csv                             sales_history.csv
item_categories.csv                            sample_data/
PBL01_h_exercise4_question.pdf                 sample_submission.csv
PBL01_i_exercise4_supplementary-documents.pdf  test.csv
PBL01_sample_code.ipynb


In [4]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#baseline

In [2]:
'''
初期設定
'''

DATA_PATH = '/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/processed_train_test_df.csv'
SAMPLESUB_PATH = 'sample_submission.csv'
SUB_PATH = '/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/submit/demand_RandomForest_Kfold.csv'


# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from statistics import mean

# データの読み込み
df = pd.read_csv(DATA_PATH)
print(df.isnull().sum())
print(df.dtypes)

# Yao-Johnson
num_cols = [
            #'sales_before_2',
            #'sales_before_3',
            #'sales_before_4',
            #'sales_before_5',
            #'sales_before_6',
            #'sales_before_7',
            #'sales_before_8',
            #'sales_before_9',
            #'sales_before_10',
            #'sales_before_11',
            #'sales_before_12',
            'sales_mean'
            ]
pt = PowerTransformer(method='yeo-johnson')
pt.fit(df[num_cols])

df[num_cols] = pt.transform(df[num_cols])

# object型の変数の取得
categories = df.columns[df.dtypes == 'object']
print(categories)

# label Encoding
for cat in categories:
    le = LabelEncoder() 
    print(cat)
    
    df[cat].fillna('missing', inplace=True)
    le = le.fit(df[cat])
    df[cat] = le.transform(df[cat])
    # LabelEncoderは数値に変換するだけであるため、最後にastype('category')としておく
    df[cat] = df[cat].astype('category') 

# trainとtestに分割
train = df[~df['sales'].isnull()]
test = df[df['sales'].isnull()]

# 説明変数と目的変数を指定
X_train = train.drop(['sales', 'price', 'category_detail'], axis=1)
Y_train = train['sales']
X_test = test.drop(['sales', 'price', 'category_detail'], axis=1)

'''
モデルの構築と評価
'''

# ライブラリのインポート
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# K分割する
folds = 3
tscv = TimeSeriesSplit(n_splits=folds)

# 各foldごとに作成したモデルごとの予測値を保存
models = []
rmses = []

for train_index, val_index in tscv.split(X_train):
    x_train = X_train.iloc[train_index]
    x_valid = X_train.iloc[val_index]
    y_train = Y_train.iloc[train_index]
    y_valid = Y_train.iloc[val_index]

    model = rf(n_estimators=50,
               random_state=1234)
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(rmse)

    models.append(model)
    rmses.append(rmse)

# 平均rmseを計算する
print(mean(rmses))

"""
予測精度：
2.1028784660653432
"""

'''
テストデータの予測
'''

# テストデータにおける予測
preds = []

for model in models:
    pred = model.predict(X_test)
    preds.append(pred)

# predsの平均を計算
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis = 0)

'''
提出
'''

# 提出用サンプルの読み込み
sub = pd.read_csv(SAMPLESUB_PATH, header=None)

# カラム1の値を置き換え
sub[1] = preds_mean

# CSVファイルの出力
sub.to_csv(SUB_PATH, header=None, index=False)

"""
スコア：
2.9039398
"""

month_bining          0
id                    0
store_id              0
sales              3060
sales_before_12       0
sales_before_11       0
sales_before_10       0
sales_before_9        0
sales_before_8        0
sales_before_7        0
sales_before_6        0
sales_before_5        0
sales_before_4        0
sales_before_3        0
sales_before_2        0
category_id           0
category              0
category_detail       0
year                  0
month                 0
price                 0
sales_mean            0
dtype: int64
month_bining       float64
id                 float64
store_id           float64
sales              float64
sales_before_12    float64
sales_before_11    float64
sales_before_10    float64
sales_before_9     float64
sales_before_8     float64
sales_before_7     float64
sales_before_6     float64
sales_before_5     float64
sales_before_4     float64
sales_before_3     float64
sales_before_2     float64
category_id          int64
category            object


'\nスコア：\n2.9039398\n'

In [ ]:
print(X_train.columns)